In [1]:
import numpy as np
import pandas as pd
from utils import get_data, get_preds, get_table, plot_rsme, other_stats, add_intervals_to_test, plot_interval_check, plot_interval_sizes
np.random.seed(2024)
train_bos, test_bos = get_data(filepath="processed_data/full_data_bos.csv", size_train=1000, size_test=1000, train_tup=(2021, 2023))
train_bos.to_csv("processed_data/train_bos.csv")
test_bos.to_csv("processed_data/test_bos.csv")
# test_bos = pd.read_csv("processed_data/test_bos.csv")

train_nyc, test_nyc = get_data(filepath="processed_data/full_data_nyc.csv", size_train=1000, size_test=100, train_tup=(2021, 2022), test_tup=(2022, 2023))
train_nyc.to_csv("processed_data/train_nyc.csv")
test_nyc.to_csv("processed_data/test_nyc.csv")
# test_nyc = pd.read_csv("processed_data/test_nyc.csv")

In [2]:
model_info = [
    # ("no_curr_pace", "stan_results/params_bos1.csv", ['total_pace', 'prop'], ['beta.1', 'beta.2'], True),
    # ("with_curr_pace", "stan_results/params_bos2.csv", ['total_pace', 'curr_pace', 'prop'], ['beta.1', 'beta.2', 'beta.3'], True),
    # ("rstan2d", "stan_results/params_bos0d.csv", ["total_pace", "curr_pace", "prop"], ["beta[1]", "beta[2]", "beta[3]"], True),
    ("no_curr_pace", "stan_results/params_nyc1.csv", ['total_pace', 'prop'], ['beta.1', 'beta.2'], True),
    ("with_curr_pace", "stan_results/params_nyc2.csv", ['total_pace', 'curr_pace', 'prop'], ['beta.1', 'beta.2', 'beta.3'], True),
]
train, test = train_nyc, test_nyc # train_bos, test_bos

models = {name: pd.read_csv(path) for (name, path, feats, betas, pleft) in model_info}
mpreds = {name: get_preds(test, pd.read_csv(path), feats_lis=feats, beta_lis=betas, propleft=pleft, name=name)
           for (name, path, feats, betas, pleft) in model_info}

test2 = get_table(test, mpreds)
test2

,id,dist,curr_pace,total_pace,finish,age,gender,year,prop,propleft,male,propxcurr,malexage,extrap,no_curr_pace,with_curr_pace
23,24651,5K,4.821601,4.821601,4.874653,42,W,2022,0.118497,0.881503,0,0.571347,0,1.587383,11.656698,11.506451
522,25150,5K,4.230118,4.230118,3.969426,32,M,2022,0.118497,0.881503,1,0.501258,32,-10.918367,2.111139,1.988405
774,25402,5K,4.051864,4.051864,3.849909,54,W,2022,0.118497,0.881503,0,0.480136,0,-9.104567,5.086219,4.976700
1064,25692,5K,3.987241,3.987241,3.748001,41,M,2022,0.118497,0.881503,1,0.472478,41,-11.258233,3.393397,3.289443
1277,25905,5K,3.671072,3.671072,3.692893,35,W,2022,0.118497,0.881503,0,0.435013,0,1.131967,18.409953,18.340788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374604,70729,40K,1.757469,1.648329,1.666667,46,W,2022,0.947980,0.052020,0,1.666045,0,4.694194,5.283935,-1.007854
374642,70767,40K,1.496110,1.669658,1.660109,51,W,2022,0.947980,0.052020,0,1.418282,0,-2.422660,-1.811987,-2.630026
374961,71086,40K,1.445087,1.628200,1.622261,68,W,2022,0.947980,0.052020,0,1.369913,0,-1.581431,-1.012645,-1.649560
375166,71291,40K,1.230921,1.614857,1.577619,53,W,2022,0.947980,0.052020,0,1.166888,0,-10.279104,-9.724886,-6.220419


In [3]:
# s_data =pd.DataFrame([[4, 0.2, .5], [3, 0.5, .75]], columns=["total_pace", "prop", "propleft"])
# get_preds(s_data, models['rstan2c'], feats_lis = ["total_pace", "prop"], beta_lis = ["beta[1]", "beta[2]"], full=True)

In [4]:
labels = ["no_curr_pace", "with_curr_pace", "extrap"]
rmse_table = plot_rsme(test2, labels, "all_errors")
rmse_table["no_curr_pace_"] = 1 - (rmse_table["no_curr_pace"] / rmse_table["extrap"])
rmse_table["with_curr_pace_"] = 1 - (rmse_table["with_curr_pace"] / rmse_table["extrap"])
rmse_table

File saved: analysis/all_errors.png


,no_curr_pace,with_curr_pace,extrap,no_curr_pace_,with_curr_pace_
dist,,,,,
5K,30.710130,30.668854,50.748153,0.394852,0.395666
10K,26.895912,26.655139,45.544882,0.409464,0.414750
15K,24.358798,24.022748,40.897712,0.404397,0.412614
20K,19.064261,18.241064,32.704128,0.417069,0.442240
25K,13.757908,13.157861,23.173161,0.406300,0.432194
30K,10.518109,9.863458,16.973292,0.380314,0.418884
35K,6.206737,5.231666,9.377434,0.338120,0.442100
40K,2.232335,1.839144,2.503979,0.108485,0.265511


In [5]:
other_stats(test2[labels], test2["finish"])

,no_curr_pace,with_curr_pace,extrap
0,19.257196,18.914524,32.231949
1,0.914944,0.917944,0.761718


In [6]:
mpreds2 = {name: (42195 / 60) / get_preds(test, pd.read_csv(path), feats_lis=feats, beta_lis=betas, propleft=pleft, name=name, full=True)
           for (name, path, feats, betas, pleft) in model_info}
# model_preds2["new1"] = (42195 / 60) / get_preds(test, pd.read_csv("stan_results/params_bos1.csv"), feats_lis=['total_pace', 'curr_pace', 'prop'], beta_lis=['beta.1', 'beta.2', 'beta.3'], propleft=True, full=True)

In [7]:
pred_names = labels[:2]
intervals_tbl = add_intervals_to_test(test2, mpreds2, pred_names)
i_check = plot_interval_check(intervals_tbl, pred_names)
i_sizes = plot_interval_sizes(intervals_tbl, pred_names)

File saved: analysis/interval_check.png
File saved: analysis/interval_sizes.png


<Figure size 4000x4000 with 0 Axes>

<Figure size 3000x2000 with 0 Axes>